<a href="https://colab.research.google.com/github/Berenice2018/DeepLearning/blob/master/FER_squeezenet_berenice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [1]:
# Imports here
#!pip install git+https://github.com/pytorch/vision

import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim

from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler

import torchvision.models as models
from torchvision.models.squeezenet import  model_urls
squeezenet1_0_state_dict = torch.hub.load_state_dict_from_url(model_urls['squeezenet1_0'])

import torchvision
from torchvision import datasets, transforms


import os,cv2
import json
import numpy as np
import pandas as pd

import time
import datetime
import copy

import matplotlib.pyplot as plt
%matplotlib inline
#from pylab import rcParams
#rcParams['figure.figsize'] = 20, 10

Downloading: "https://download.pytorch.org/models/squeezenet1_0-a815701f.pth" to /root/.cache/torch/checkpoints/squeezenet1_0-a815701f.pth
100%|██████████| 5017600/5017600 [00:00<00:00, 16207423.76it/s]


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
fer= "./gdrive/My Drive/Colab Notebooks/Fer-dataset/fer2013/fer2013.csv"
ferr= "./gdrive/My Drive/Colab Notebooks/Fer-dataset"

#!ls   '/content/gdrive/My Drive/Colab Notebooks/Fer-dataset/fer2013/'

fer2013.bib  fer2013.csv  PrivateTest  PublicTest  README  Training


In [0]:
!pip install mxnet

### Convert and save images to Gdrive

In [0]:
#! /usr/bin/env python3
# -*-coding: utf-8-*-

__author__ = 'Moonkie'

import numpy as np
import cv2
import mxnet as mx
import pandas as pd
import random
import os

curdir = os.path.abspath(os.path.dirname(fer))

def gen_record(csvfile,channel):
    data = pd.read_csv(csvfile,delimiter=',',dtype='a')
    labels = np.array(data['emotion'],np.float)
    # print(labels,'\n',data['emotion'])
        
    imagebuffer = np.array(data['pixels'])
    images = np.array([np.fromstring(image,np.uint8,sep=' ') for image in imagebuffer])
    del imagebuffer
    num_shape = int(np.sqrt(images.shape[-1]))
    images.shape = (images.shape[0],num_shape,num_shape)
    # img=images[0];cv2.imshow('test',img);cv2.waitKey(0);cv2.destroyAllWindow();exit()
    dirs = set(data['Usage'])
    subdirs = set(labels)
    class_dir = {}
    for dr in dirs:
        dest = os.path.join(curdir,dr)
        class_dir[dr] = dest
        if not os.path.exists(dest):
            os.mkdir(dest)
            
    data = zip(labels,images,data['Usage'])
    
    for d in data:
        destdir = os.path.join(class_dir[d[-1]],str(int(d[0])))
        if not os.path.exists(destdir):
            os.mkdir(destdir)
        img = d[1]
        filepath = unique_name(destdir,d[-1])
        print('[^_^] Write image to %s' % filepath)
        if not filepath:
            continue
        sig = cv2.imwrite(filepath,img)
        if not sig:
            print('Error')
            exit(-1)


def unique_name(pardir,prefix,suffix='jpg'):
    filename = '{0}_{1}.{2}'.format(prefix,random.randint(1,10**8),suffix)
    filepath = os.path.join(pardir,filename)
    if not os.path.exists(filepath):
        return filepath
    unique_name(pardir,prefix,suffix)
    


if __name__ == '__main__':
    filename = 'fer2013.csv'
    filename = os.path.join(curdir,filename)
    gen_record(filename,1)
    
    # ##################### test
    # tmp = unique_name('./Training','Training')
    # print(tmp)

### Load data , image transform

In [4]:

data_dir = "./gdrive/My Drive/Colab Notebooks/Fer-dataset/fer2013"
train_dir = data_dir + '/Training'
valid_dir = data_dir + '/PublicTest'
test_dir = data_dir + '/PrivateTest'

# original img size = 48 x48 px
# TODO: Define your transforms for the training and validation sets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        #transforms.CenterCrop(256),
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])
    }

# TODO: Load the datasets with ImageFolder

dirs = {'train': train_dir, 
        'valid': valid_dir,
        'test': test_dir}

datasets = {x: torchvision.datasets.ImageFolder(dirs[x], transform=data_transforms[x]) for x in ['train', 'valid', 'test']}

dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=32, shuffle=True, num_workers=7) for x in ['train', 'valid','test']}

dataset_sizes = {x: len(datasets[x]) 
                              for x in ['train', 'valid','test']}
print(dataset_sizes)

{'train': 32556, 'valid': 3589, 'test': 3589}


In [0]:
class_names = datasets['train'].classes
print(class_names)

['0', '1', '2', '3', '4', '5', '6']


### Show some sample images

In [0]:
# get a batch of training images
dataiter = iter(dataloaders['train'])
images, _ = dataiter.next()
images = images.numpy() # convert images to numpy

# plot the images of the batch, along with the corresponding labels
fig = plt.figure(figsize=(20, 4))

for idx in np.arange(16):
    ax = fig.add_subplot(2, 16/2, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))  # convert from Tensor image

### Model train helper

In [0]:
def get_time():
      hour = datetime.datetime.today().hour +2
      minute = datetime.datetime.today().minute
      second = datetime.datetime.today().second
      return hour, minute, second
    


def weights_init_normal(m):
    '''Takes in a module and initializes all linear layers with weight
       values taken from a normal distribution.'''
    classname = m.__class__.__name__
    # for every Linear layer in a model
    if classname.find('Linear') != -1:
        n = m.in_features
        # m.weight.data shoud be taken from a normal distribution
        m.weight.data.normal_(0, 1/np.sqrt(n))
        # m.bias.data should be 0
        m.bias.data.fill_(0)

In [0]:
def train_model(model, name, criteria, optimizer, scheduler = None, epochs=50, device='cuda'):
    h, m, s = get_time()
    print('starting the training at {}:{}:{}'.format(h, m, s))
    since = time.time()
    best_model = copy.deepcopy(model.state_dict())
    
    # initialize tracker for accuracy 
    min_loss = np.Inf
    TS_loss = pd.DataFrame(np.nan, index = range(1,epochs+1), columns = ['train loss','valid loss'])
    TS_acc = pd.DataFrame(np.nan,index = range(1,epochs+1), columns = ['train acc','valid acc'])
    
    for epoch in range(1, epochs+1):        
        # monitor loss/accuracy
        train_loss_running = 0.0
        valid_loss_running = 0.0
        train_acc_running = 0.0
        valid_acc_running = 0.0

        ###################
        # train the model #
        ###################
        if scheduler:
            scheduler.step()
        model.train() # prep model for training
        
        for data, target in dataloaders['train']:
    
            # Move input and data tensors to the default device
            data, target = data.to(device), target.to(device)
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criteria(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss 
            train_loss_running += loss.item()*data.size(0)
            # accuracy
            _, preds = torch.max(output, 1)
            train_acc_running  += torch.sum(preds == target.data)
    

        ######################    
        # validate the model #
        ######################
        model.eval()
        with torch.no_grad():
            for data, target in dataloaders['valid']:
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criteria(output, target)
                valid_loss_running += loss.item()*data.size(0)
                
                _, preds = torch.max(output, 1)
                valid_acc_running  += torch.sum(preds == target.data)
                #print('valid_acc_running {:.6f}'.format(valid_acc_running))
                
                # Calculate accuracy
                #ps = torch.exp(logps)
                #top_p, top_class = ps.topk(1, dim=1)
                #equals = top_class == labels.view(*top_class.shape)
                #valid_loss_running += torch.mean(equals.type(torch.FloatTensor)).item()

        # statistics
        train_loss = train_loss_running/ dataset_sizes['train']
        valid_loss = valid_loss_running/ dataset_sizes['valid']
        train_acc = train_acc_running.double()/ dataset_sizes['train']
        valid_acc = valid_acc_running.double()/ dataset_sizes['valid']
        
        TS_loss.loc[epoch] = [train_loss, valid_loss]
        TS_acc.loc[epoch] = [(train_acc.cpu().numpy()), (valid_acc.cpu().numpy())]                                         

        # print training/validation statistics 
        hour, minute, second = get_time()
        print('Epoch: {} at {}:{}:{} \tTrain. Loss: {:.6f} \tValid. Loss: {:.6f} \t Accur.: {:.10f}'.format(
                  epoch,
                  hour, minute, second,
                  train_loss,
                  valid_loss,
                  valid_acc 
        ))
                                                             
        # save model if validation acc has increased
        # BC: using the lower validation loss as criterium, when saving a model
        if valid_loss <= min_loss:
            old_loss_min = min_loss
            min_loss = valid_loss
            save = True
            
            print('*** Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            old_loss_min, min_loss))        
    
    #End
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    #print('Best Valid Acc: {:4f}'.format(best_acc))
                                                   
    TS_loss.plot(title = 'loss')
    TS_acc.plot(title = 'acc')
                                                   
    # load best model weights
    model.load_state_dict(best_model)
    torch.save(model.state_dict(), name)
    
    return model

### Hyper params, start training

In [7]:
#model = torch.hub.load_state_dict_from_url(model_urls['squeezenet1_0'])
#model = torch.hub.load('pytorch/vision', 'squeezenet1_0', pretrained=True)

no_of_classes = 7
transfermodel = models.squeezenet1_0(pretrained =True)

for param in transfermodel.parameters():
    param.requires_grad = False
    
transfermodel.classifier[1] = nn.Conv2d(512, no_of_classes, kernel_size=(1,1), stride=(1,1))

#transfermodel.apply(weights_init_normal)

#transfermodel.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transfermodel.to(device);

transfermodel

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (5): Fire(
      (squeeze): Conv2d(128, 32, kerne

In [0]:
epochs = 15


criteria = nn.CrossEntropyLoss()    
lr = 5e-3 #1e-2 #1e-3
optimizer = optim.Adam(transfermodel.parameters(), lr=lr)

stepsize = 4
#scheduler = lr_scheduler.StepLR(optimizer, step_size=stepsize, gamma=0.1)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [4, 7, 10, 12], 0.2)


# create model name
save_path = './gdrive/My Drive/Colab Notebooks/Fer-dataset/'
modelname = '{}squeeze1_0-{}-lr{}-{}{}-epo{}.pth'.format(save_path, 'Adam', lr, 'MultiStep4_7_10_12', stepsize, epochs)

print(modelname)

model_ft = train_model(transfermodel, modelname, criteria, optimizer, scheduler, epochs, device)



./gdrive/My Drive/Colab Notebooks/Fer-dataset/squeeze1_0-Adam-lr0.005-MultiStep4_7_10_124-epo15.pth
starting the training at 10:12:30
Epoch: 1 at 10:42:16 	Train. Loss: 1.513116 	Valid. Loss: 1.511674 	 Accur.: 0.4541655057
*** Validation loss decreased (inf --> 1.511674).  Saving model ...
Epoch: 2 at 10:44:53 	Train. Loss: 1.448081 	Valid. Loss: 1.468687 	 Accur.: 0.4745054333
*** Validation loss decreased (1.511674 --> 1.468687).  Saving model ...
Epoch: 3 at 10:47:29 	Train. Loss: 1.431797 	Valid. Loss: 1.503380 	 Accur.: 0.4714405127
Epoch: 4 at 10:50:7 	Train. Loss: 1.387956 	Valid. Loss: 1.445675 	 Accur.: 0.4842574533
*** Validation loss decreased (1.468687 --> 1.445675).  Saving model ...
Epoch: 5 at 10:52:44 	Train. Loss: 1.382197 	Valid. Loss: 1.435690 	 Accur.: 0.4831429368
*** Validation loss decreased (1.445675 --> 1.435690).  Saving model ...
Epoch: 6 at 10:55:21 	Train. Loss: 1.372908 	Valid. Loss: 1.423354 	 Accur.: 0.4878796322
*** Validation loss decreased (1.435690 

### Transfer learning, unfreeze layers

In [0]:
base_path = './gdrive/My Drive/Colab Notebooks/Fer-dataset/'
#modelname = base_path +'resnext50-Adam-lr0.03-StepLR_4-epo15.pth'

modely = models.resnext50_32x4d(pretrained=True)

#modely.load_state_dict(torch.load(modelname),strict=False)


In [0]:
print(modely)


In [0]:
for param in modely.parameters():
    param.requires_grad = False
    
fc_in = modely.fc.in_features

transferclassifier = nn.Sequential(
                        nn.BatchNorm1d(fc_in),
                        nn.Linear(fc_in, 7),
                        #nn.ReLU(),
                        #nn.Linear(1024, 133)
                        )

#transfermodel.fc = transferclassifier # resnet
modely.fc = transferclassifier
modely.apply(weights_init_normal)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modely.to(device);


In [0]:

criterion_tf = nn.CrossEntropyLoss()  

lr_tf = 1e-3
optimizer_tf = optim.Adam(modely.parameters(), lr=lr_tf)

stepsize_tf = 7
scheduler_tf = lr_scheduler.StepLR(optimizer_tf, step_size=stepsize_tf, gamma=0.1)

epochs_ft = 20

# create model name
save_path = './gdrive/My Drive/Colab Notebooks/Fer-dataset/'
modelname_xy = '{}tf_resnext50-{}-lr{}-{}{}-epo{}.pth'.format(
    save_path, 
    'Adam', 
    lr_tf, 
    'StepLR_', 
    stepsize_tf, 
    epochs_ft
)

print(modelname_xy)

modelxy = train_model(
    modely, 
    modelname_xy, 
    criterion_tf, 
    optimizer_tf, 
    scheduler_tf, 
    epochs_ft, 
    device
)

./gdrive/My Drive/Colab Notebooks/Fer-dataset/tf_resnext50-Adam-lr0.001-StepLR_7-epo20.pth
starting the training at 19:27:37
Epoch: 1 at 19:42:33 	Train. Loss: 1.577323 	Valid. Loss: 1.562634 	 Accur.: 0.4001114517
*** Validation loss decreased (inf --> 1.562634).  Saving model ...
Epoch: 2 at 19:47:58 	Train. Loss: 1.507071 	Valid. Loss: 1.544317 	 Accur.: 0.4104207300
*** Validation loss decreased (1.562634 --> 1.544317).  Saving model ...
Epoch: 3 at 19:53:23 	Train. Loss: 1.481276 	Valid. Loss: 1.518490 	 Accur.: 0.4271384787
*** Validation loss decreased (1.544317 --> 1.518490).  Saving model ...
Epoch: 4 at 19:58:49 	Train. Loss: 1.468990 	Valid. Loss: 1.524944 	 Accur.: 0.4240735581
Epoch: 5 at 20:4:14 	Train. Loss: 1.456688 	Valid. Loss: 1.507987 	 Accur.: 0.4329896907
*** Validation loss decreased (1.518490 --> 1.507987).  Saving model ...
Epoch: 6 at 20:9:40 	Train. Loss: 1.448987 	Valid. Loss: 1.507886 	 Accur.: 0.4335469490
*** Validation loss decreased (1.507987 --> 1.5078

KeyboardInterrupt: ignored

### Test

In [0]:
def test(loader, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    
    print('test started at ', get_time())
    model.eval()
    for batch_idx, (data, target) in enumerate(loader):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('test finished at ', get_time())
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [0]:

# call test function    
test(dataloaders['test'], model_ft, criteria, device)

test started at  (25, 11, 34)
test finished at  (25, 14, 21)
Test Loss: 1.965151


Test Accuracy: 17% (633/3589)


### Unused stuff ?

In [0]:
# save model as a checkpoint file after every 5 epochs
def save_model_checkpoint(model, optimizer, criteria, epochs):
  model.to('cpu')
  model.class_to_idx = datasets['train'].class_to_idx
  checkpoint = {'input_size': 48*48*1,
                'output_size': 7,
                'model': model,
                'state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'criterion': criteria,
                'epochs': epochs,
                'class_to_idx': model.class_to_idx}

  path = "/content/gdrive/My Drive/facial_expressions.pth"
  torch.save(checkpoint,path)


# load model function 
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath, map_location='cpu')
    model = checkpoint["model"]
    model.load_state_dict(checkpoint['state_dict'])
    return model

In [0]:
#! /usr/bin/env python3
# -*-coding: utf-8-*-

__author__ = 'Moonkie'

import numpy as np
import cv2
import mxnet as mx
import pandas as pd
import random
import os

curdir = os.path.abspath(os.path.dirname(fer))

def gen_record(csvfile,channel):
    data = pd.read_csv(csvfile,delimiter=',',dtype='a')
    labels = np.array(data['emotion'],np.float)
    # print(labels,'\n',data['emotion'])
        
    imagebuffer = np.array(data['pixels'])
    images = np.array([np.fromstring(image,np.uint8,sep=' ') for image in imagebuffer])
    del imagebuffer
    num_shape = int(np.sqrt(images.shape[-1]))
    images.shape = (images.shape[0],num_shape,num_shape)
    # img=images[0];cv2.imshow('test',img);cv2.waitKey(0);cv2.destroyAllWindow();exit()
    dirs = set(data['Usage'])
    subdirs = set(labels)
    class_dir = {}
    for dr in dirs:
        dest = os.path.join(curdir,dr)
        class_dir[dr] = dest
        if not os.path.exists(dest):
            os.mkdir(dest)
            
    data = zip(labels,images,data['Usage'])
    
    for d in data:
        destdir = os.path.join(class_dir[d[-1]],str(int(d[0])))
        if not os.path.exists(destdir):
            os.mkdir(destdir)
        img = d[1]
        filepath = unique_name(destdir,d[-1])
        print('[^_^] Write image to %s' % filepath)
        if not filepath:
            continue
        sig = cv2.imwrite(filepath,img)
        if not sig:
            print('Error')
            exit(-1)


def unique_name(pardir,prefix,suffix='jpg'):
    filename = '{0}_{1}.{2}'.format(prefix,random.randint(1,10**8),suffix)
    filepath = os.path.join(pardir,filename)
    if not os.path.exists(filepath):
        return filepath
    unique_name(pardir,prefix,suffix)
    


if __name__ == '__main__':
    filename = 'fer2013.csv'
    filename = os.path.join(curdir,filename)
    gen_record(filename,1)
    
    # ##################### test
    # tmp = unique_name('./Training','Training')
    # print(tmp)